In [78]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt

from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.regression import MapieRegressor
from mapie.subsample import Subsample

from sklearn.preprocessing import  MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [79]:
path = 'data/data.csv'
data = pd.read_csv(path)
normalized_pctg_change = data['normalized_percent_change'] # Save variable fot later use in model
prediction = data['prediction'] 
data.drop(columns=['normalized_percent_change'], inplace=True)

In [80]:
xgboost_df = data.copy()
# One-hot encoding
demographic_vars = ['gender_source_value', 'race_source_value', 'ethnicity_source_value']
xgboost_df = pd.get_dummies(xgboost_df, columns=demographic_vars)
# Scaling: Apparently there's no difference if a use a StandardScaler vs MinMaxScaler
scaler = MinMaxScaler()
numeric_vars = ['mean_led_per_visit', 'age', 'length_of_stay', 'days_since_last_visit', 'days_to_diagnosis']
for i in range(len(numeric_vars)):
    xgboost_df[numeric_vars[i]] = scaler.fit_transform(xgboost_df[[numeric_vars[i]]])

# Reordering the columns so that the target variable is the last one
prediction_to_last = xgboost_df.pop('prediction')
xgboost_df['prediction'] = prediction_to_last

# Defining the features and target variable
X = xgboost_df.iloc[:, :-1] # Shape is rows x features (38)
y = xgboost_df.iloc[:, -1]

# Split data into training and testing sets
# I don't need to use cupy or cudf to send the data to the GPU because I'm using DMatrix
random_state = 21
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
d_train = xgb.DMatrix(X_train, y_train, label=y_train)
d_test = xgb.DMatrix(X_test, y_test, label=y_test)
best_params = {
    # Classification
    "eval_metric": "auc", # Area under the curve
    "objective": "binary:logistic", # Logistic regression for binary classification, output probability
    'sampling_method': 'gradient_based', # The selection probability for each training instance is proportional to the regularized absolute value of gradients 
    'alpha': 0.1, # L1 regularization
    'lambda': 1, # L2 regularization
    'learning_rate': 0.1, 
    'max_depth': 7,
    'num_boost_round': 700, 
    'tree_method': 'hist', 
    'device': "cuda",
}
num_boost_round = best_params['num_boost_round']
# Prior AUC: 0.86066216116602046
# Best new = 0.92090948515188154 difference is nwq 
regression_params_short = {'alpha': 0.1, 'lambda': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 700, "eval_metric": 'rmse', 'objective': 'reg:squarederror', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda"} # Updated 28/08/2024. Range (0,1)
regression_params_long = {'alpha': 1, 'lambda': 0.1, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 700, "eval_metric": 'rmse', 'objective': 'reg:squarederror', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda"} # Updated 01/09/2024. Range (-1,1) Decreases coverage in Jackknife+ by 0.02
binary_params = {'alpha': 0, 'lambda': 0.1, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 800, "eval_metric": 'auc', 'objective': 'binary:logistic', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda",} 

d_all = xgb.DMatrix(X)
model = xgb.train(best_params, d_train, num_boost_round=num_boost_round, evals=((d_test, "test"),),verbose_eval=True, early_stopping_rounds=10)
y_pred_proba_all = model.predict(d_all, iteration_range=(0, model.best_iteration + 1))
y_pred_all = (y_pred_proba_all > 0.5).astype(int)

[0]	test-auc:0.86294
[1]	test-auc:0.86382
[2]	test-auc:0.87933
[3]	test-auc:0.88895
[4]	test-auc:0.89025
[5]	test-auc:0.89470
[6]	test-auc:0.89596
[7]	test-auc:0.89800
[8]	test-auc:0.89995
[9]	test-auc:0.90051
[10]	test-auc:0.90198
[11]	test-auc:0.90364
[12]	test-auc:0.90498
[13]	test-auc:0.90394
[14]	test-auc:0.90669
[15]	test-auc:0.90791
[16]	test-auc:0.91025
[17]	test-auc:0.91106
[18]	test-auc:0.91245
[19]	test-auc:0.91457
[20]	test-auc:0.91526
[21]	test-auc:0.91669
[22]	test-auc:0.91825
[23]	test-auc:0.91827
[24]	test-auc:0.91895
[25]	test-auc:0.91995
[26]	test-auc:0.92116
[27]	test-auc:0.92150
[28]	test-auc:0.92219
[29]	test-auc:0.92224
[30]	test-auc:0.92256
[31]	test-auc:0.92388
[32]	test-auc:0.92452
[33]	test-auc:0.92480
[34]	test-auc:0.92626
[35]	test-auc:0.92725
[36]	test-auc:0.92796
[37]	test-auc:0.92874
[38]	test-auc:0.92923
[39]	test-auc:0.92959
[40]	test-auc:0.92964
[41]	test-auc:0.92987
[42]	test-auc:0.93013
[43]	test-auc:0.93105
[44]	test-auc:0.93176
[45]	test-auc:0.9320

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:56:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[48]	test-auc:0.93430
[49]	test-auc:0.93468
[50]	test-auc:0.93605
[51]	test-auc:0.93615
[52]	test-auc:0.93664
[53]	test-auc:0.93707
[54]	test-auc:0.93782
[55]	test-auc:0.93816
[56]	test-auc:0.93814
[57]	test-auc:0.93839
[58]	test-auc:0.93877
[59]	test-auc:0.93946
[60]	test-auc:0.93984
[61]	test-auc:0.94022
[62]	test-auc:0.94035
[63]	test-auc:0.94063
[64]	test-auc:0.94162
[65]	test-auc:0.94183
[66]	test-auc:0.94225
[67]	test-auc:0.94252
[68]	test-auc:0.94300
[69]	test-auc:0.94326
[70]	test-auc:0.94375
[71]	test-auc:0.94465
[72]	test-auc:0.94495
[73]	test-auc:0.94504
[74]	test-auc:0.94561
[75]	test-auc:0.94609
[76]	test-auc:0.94658
[77]	test-auc:0.94715
[78]	test-auc:0.94728
[79]	test-auc:0.94766
[80]	test-auc:0.94787
[81]	test-auc:0.94791
[82]	test-auc:0.94912
[83]	test-auc:0.94931
[84]	test-auc:0.94967
[85]	test-auc:0.94969
[86]	test-auc:0.94961
[87]	test-auc:0.94970
[88]	test-auc:0.94978
[89]	test-auc:0.94994
[90]	test-auc:0.95027
[91]	test-auc:0.95048
[92]	test-auc:0.95081
[93]	test-

In [81]:
def split_into_five_equal_parts(X, y_binary, y_continuous, random_state=42):
    """
    Split data into 5 equal parts, maintaining alignment between binary and continuous outcomes
    
    Parameters:
    -----------
    X : features DataFrame
    y_binary : binary outcome (change/no change in LEDD)
    y_continuous : continuous outcome (normalized percentage change)
    random_state : for reproducibility
    
    Returns:
    --------
    Training, calibration, and test sets for both binary and continuous outcomes
    """
    # First split: 80% for train+calibration, 20% for test
    X_temp, X_test, y_binary_temp, y_binary_test, y_cont_temp, y_cont_test = train_test_split(
        X, y_binary, y_continuous, test_size=0.2, random_state=random_state
    )
    
    # Split the remaining 80% into 4 equal parts
    X_train, X_temp2, y_binary_train, y_binary_temp2, y_cont_train, y_cont_temp2 = train_test_split(
        X_temp, y_binary_temp, y_cont_temp, test_size=0.75, random_state=random_state
    )
    
    # Split the 60% into 3 equal calibration sets
    X_calib1, X_temp3, y_binary_calib1, y_binary_temp3, y_cont_calib1, y_cont_temp3 = train_test_split(
        X_temp2, y_binary_temp2, y_cont_temp2, test_size=0.666, random_state=random_state
    )
    
    X_calib2, X_calib3, y_binary_calib2, y_binary_calib3, y_cont_calib2, y_cont_calib3 = train_test_split(
        X_temp3, y_binary_temp3, y_cont_temp3, test_size=0.5, random_state=random_state
    )
    
    return (
        X_train, X_calib1, X_calib2, X_calib3, X_test,
        y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
        y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test
    )

In [82]:
def conformal_prediction_zero_inflated(X, y_binary, y_continuous, alpha_tilda=0.9):
    # Split data into five parts
    (X_train, X_calib1, X_calib2, X_calib3, X_test,
     y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
     y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test) = split_into_five_equal_parts(
        X, y_binary, y_continuous
    )
    
    # Train classification model on binary outcome
    classifier = xgb.XGBClassifier(**binary_params)
    classifier.fit(X_train, y_binary_train)
    
    # Train regression model on non-zero continuous outcomes
    mask_nonzero = y_binary_train == 1
    regressor = xgb.XGBRegressor(**regression_params_short)
    regressor.fit(X_train[mask_nonzero], y_cont_train[mask_nonzero])
    
    # First calibration set: Determine alpha_r
    probs_calib1 = classifier.predict_proba(X_calib1)[:, 1]
    r_values = np.arange(0.1, 0.95, 0.05)
    all_interval_lengths = []
    
    for r in r_values:
        alpha_r = np.quantile(probs_calib1, r)
        
        # Second calibration set: Calculate beta_hat
        probs_calib2 = classifier.predict_proba(X_calib2)[:, 1]
        pred_zeros = probs_calib2 <= alpha_r
        beta_hat = np.mean(y_binary_calib2[pred_zeros] == 0)
        
        # Calculate final quantile
        final_quantile = (alpha_tilda - beta_hat * r) / (1 - r)
        if final_quantile < 0 or final_quantile > 1:
            continue
            
        # Third calibration set: Calculate interval width
        probs_calib3 = classifier.predict_proba(X_calib3)[:, 1]
        nonzero_mask = probs_calib3 > alpha_r
        if not any(nonzero_mask):
            continue
            
        y_pred_nonzero = regressor.predict(X_calib3[nonzero_mask])
        residuals = np.abs(y_cont_calib3[nonzero_mask] - y_pred_nonzero)
        interval_width = np.quantile(residuals, final_quantile)
        all_interval_lengths.append((r, interval_width, alpha_r))
    
    # Choose best r value and corresponding width
    best_r, best_width, best_alpha_r = min(all_interval_lengths, key=lambda x: x[1])
    
    # Make predictions on test set
    test_probs = classifier.predict_proba(X_test)[:, 1]
    test_pred = regressor.predict(X_test)
    
    # Create prediction intervals
    lower_bound = np.zeros_like(test_pred)
    upper_bound = np.zeros_like(test_pred)
    nonzero_mask = test_probs > best_alpha_r
    
    lower_bound[nonzero_mask] = test_pred[nonzero_mask] - best_width
    upper_bound[nonzero_mask] = test_pred[nonzero_mask] + best_width
    
    return lower_bound, upper_bound, y_cont_test

In [83]:
lower_bound, upper_bound, y_test = conformal_prediction_zero_inflated(
    X=X,
    y_binary=y_pred_all,
    y_continuous=normalized_pctg_change,
    alpha_tilda=0.9  # Target coverage (90%)
)

# Calculate and print coverage
coverage = np.mean((y_test >= lower_bound) & (y_test <= upper_bound))
interval_width = np.mean(upper_bound - lower_bound)
print(f"Coverage: {coverage:.3f}")
print(f"Average interval width: {interval_width:.3f}")

Coverage: 0.845
Average interval width: 0.139


In [89]:
def conformal_prediction_zero_inflated_mapie(X, y_binary, y_continuous, alpha_tilda=0.9):
    """
    Two-stage conformal prediction using MAPIE for regression intervals.
    First stage: XGBoost classifier to predict zero/non-zero
    Second stage: MAPIE with XGBoost regressor for prediction intervals
    """
    # Split data into five parts
    (X_train, X_calib1, X_calib2, X_calib3, X_test,
     y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
     y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test) = split_into_five_equal_parts(
        X, y_binary, y_continuous
    )
    
    # First stage: Classification model
    classifier = xgb.XGBClassifier(**binary_params)
    classifier.fit(X_train, y_binary_train)
    
    # Get probabilities for calibration set 1
    probs_calib1 = classifier.predict_proba(X_calib1)[:, 1]
    
    # Try different r values
    r_values = np.arange(0.1, 0.95, 0.05)
    all_interval_lengths = []
    
    # Initialize MAPIE with XGBoost regressor
    regressor = xgb.XGBRegressor(**regression_params_short)
    mapie = MapieRegressor(
        estimator=regressor,
        method="plus",  # Jack-knife+ method
        cv="prefit",
        random_state=42
    )
    
    # Fit regressor on non-zero training data
    mask_nonzero = y_binary_train == 1
    regressor.fit(X_train[mask_nonzero], y_cont_train[mask_nonzero])
    
    for r in r_values:
        # First calibration: threshold for zero/non-zero
        alpha_r = np.quantile(probs_calib1, r)
        
        # Second calibration: accuracy of zero predictions
        probs_calib2 = classifier.predict_proba(X_calib2)[:, 1]
        pred_zeros = probs_calib2 <= alpha_r
        beta_hat = np.mean(y_binary_calib2[pred_zeros] == 0)
        
        # Calculate final quantile
        final_quantile = (alpha_tilda - beta_hat * r) / (1 - r)
        if final_quantile < 0 or final_quantile > 1:
            continue
            
        # Third calibration: Use MAPIE for interval width
        probs_calib3 = classifier.predict_proba(X_calib3)[:, 1]
        nonzero_mask = probs_calib3 > alpha_r
        if not any(nonzero_mask):
            continue
        
        # Fit MAPIE on non-zero calibration data
        X_calib3_nonzero = X_calib3[nonzero_mask]
        y_calib3_nonzero = y_cont_calib3[nonzero_mask]
        
        # Get MAPIE prediction intervals
        mapie.fit(X_calib3_nonzero, y_calib3_nonzero)
        _, y_pis = mapie.predict(X_calib3_nonzero, alpha=final_quantile)
        
        # Calculate interval width as mean of MAPIE intervals
        interval_width = np.mean(y_pis[:, 1, 0] - y_pis[:, 0, 0])
        all_interval_lengths.append((r, interval_width, alpha_r))
    
    if not all_interval_lengths:
        raise ValueError("No valid intervals found. Try adjusting r_values or alpha_tilda.")
    
    # Choose best r value
    best_r, best_width, best_alpha_r = min(all_interval_lengths, key=lambda x: x[1])
    
    # Final predictions on test set
    test_probs = classifier.predict_proba(X_test)[:, 1]
    nonzero_mask_test = test_probs > best_alpha_r
    
    # Initialize bounds
    lower_bound = np.zeros(len(X_test))
    upper_bound = np.zeros(len(X_test))
    
    if any(nonzero_mask_test):
        # Get MAPIE predictions for non-zero cases
        _, y_pis_test = mapie.predict(X_test[nonzero_mask_test], alpha=0.8)
        lower_bound[nonzero_mask_test] = y_pis_test[:, 0, 0]
        upper_bound[nonzero_mask_test] = y_pis_test[:, 1, 0]
    
    return lower_bound, upper_bound, y_cont_test

# Usage
lower_bound_mapie, upper_bound_mapie, y_test_mapie = conformal_prediction_zero_inflated_mapie(
    X=X,
    y_binary=y_pred_all,
    y_continuous=normalized_pctg_change,
    alpha_tilda=0.9
)

# Calculate coverage and interval width
coverage_mapie = np.mean((y_test_mapie >= lower_bound_mapie) & (y_test_mapie <= upper_bound_mapie))
interval_width_mapie = np.mean(upper_bound_mapie - lower_bound_mapie)
print(f"MAPIE Coverage: {coverage_mapie:.3f}")
print(f"MAPIE Average interval width: {interval_width_mapie:.3f}")

MAPIE Coverage: 0.735
MAPIE Average interval width: 0.003


In [87]:
def conformal_prediction_zero_inflated_mapie(X, y_binary, y_continuous, alpha_tilda=0.9):
    """
    Two-stage conformal prediction using MAPIE for regression intervals.
    """
    # Split data into five parts
    (X_train, X_calib1, X_calib2, X_calib3, X_test,
     y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
     y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test) = split_into_five_equal_parts(
        X, y_binary, y_continuous
    )
    
    # First stage: Classification model
    classifier = xgb.XGBClassifier(**binary_params)
    classifier.fit(X_train, y_binary_train)
    
    # Get probabilities for calibration set 1
    probs_calib1 = classifier.predict_proba(X_calib1)[:, 1]
    
    # Try different r values
    r_values = np.arange(0.1, 0.95, 0.05)
    all_interval_lengths = []
    
    # Initialize MAPIE with XGBoost regressor
    regressor = xgb.XGBRegressor(**regression_params_short)
    
    # Fit regressor on non-zero training data
    mask_nonzero = y_binary_train == 1
    regressor.fit(X_train[mask_nonzero], y_cont_train[mask_nonzero])
    
    # Initialize MAPIE with the fitted regressor
    mapie = MapieRegressor(
        estimator=regressor,
        method="plus",
        cv="prefit",
        random_state=42
    )
    
    for r in r_values:
        # First calibration: threshold for zero/non-zero
        alpha_r = np.quantile(probs_calib1, r)
        
        # Second calibration: accuracy of zero predictions
        probs_calib2 = classifier.predict_proba(X_calib2)[:, 1]
        pred_zeros = probs_calib2 <= alpha_r
        beta_hat = np.mean(y_binary_calib2[pred_zeros] == 0)
        
        # Calculate final quantile and ensure it's in [0,1]
        final_quantile = (alpha_tilda - beta_hat * r) / (1 - r)
        final_quantile = np.clip(final_quantile, 0.01, 0.99)  # Clip to valid range
        
        # Third calibration: Use MAPIE for interval width
        probs_calib3 = classifier.predict_proba(X_calib3)[:, 1]
        nonzero_mask = probs_calib3 > alpha_r
        if not any(nonzero_mask):
            continue
        
        # Fit MAPIE on non-zero calibration data
        X_calib3_nonzero = X_calib3[nonzero_mask]
        y_calib3_nonzero = y_cont_calib3[nonzero_mask]
        
        try:
            # Get MAPIE prediction intervals
            mapie.fit(X_calib3_nonzero, y_calib3_nonzero)
            _, y_pis = mapie.predict(X_calib3_nonzero, alpha=final_quantile)
            
            # Calculate interval width as mean of MAPIE intervals
            interval_width = np.mean(y_pis[:, 1, 0] - y_pis[:, 0, 0])
            all_interval_lengths.append((r, interval_width, alpha_r))
        except Exception as e:
            print(f"Skipping r={r} due to error: {e}")
            continue
    
    if not all_interval_lengths:
        raise ValueError("No valid intervals found. Try adjusting r_values or alpha_tilda.")
    
    # Choose best r value
    best_r, best_width, best_alpha_r = min(all_interval_lengths, key=lambda x: x[1])
    
    # Final predictions on test set
    test_probs = classifier.predict_proba(X_test)[:, 1]
    nonzero_mask_test = test_probs > best_alpha_r
    
    # Initialize bounds
    lower_bound = np.zeros(len(X_test))
    upper_bound = np.zeros(len(X_test))
    
    if any(nonzero_mask_test):
        # Get MAPIE predictions for non-zero cases
        _, y_pis_test = mapie.predict(X_test[nonzero_mask_test], alpha=0.9)  # Use fixed alpha for final prediction
        lower_bound[nonzero_mask_test] = y_pis_test[:, 0, 0]
        upper_bound[nonzero_mask_test] = y_pis_test[:, 1, 0]
    
    return lower_bound, upper_bound, y_cont_test

In [88]:
lower_bound_mapie, upper_bound_mapie, y_test_mapie = conformal_prediction_zero_inflated_mapie(
    X=X,
    y_binary=y_pred_all,
    y_continuous=normalized_pctg_change,
    alpha_tilda=0.9
)

# Calculate coverage and interval width
coverage_mapie = np.mean((y_test_mapie >= lower_bound_mapie) & (y_test_mapie <= upper_bound_mapie))
interval_width_mapie = np.mean(upper_bound_mapie - lower_bound_mapie)
print(f"MAPIE Coverage: {coverage_mapie:.3f}")
print(f"MAPIE Average interval width: {interval_width_mapie:.3f}")

MAPIE Coverage: 0.737
MAPIE Average interval width: 0.001


In [91]:
from matplotlib.ticker import FormatStrFormatter
from matplotlib.offsetbox import AnnotationBbox, TextArea

round_to = 2

def sort_y_values(y_test, y_pred, y_pis):
    """
    Sorting the dataset in order to make plots using the fill_between function.
    """
    indices = np.argsort(y_test)
    y_test_sorted = np.array(y_test)[indices]
    y_pred_sorted = y_pred[indices]
    y_lower_bound = y_pis[:, 0, 0][indices]
    y_upper_bound = y_pis[:, 1, 0][indices]
    return y_test_sorted, y_pred_sorted, y_lower_bound, y_upper_bound


def plot_prediction_intervals(
    title,
    axs,
    y_test_sorted,
    y_pred_sorted,
    lower_bound,
    upper_bound,
    coverage,
    width,
    num_plots_idx
):
    """
    Plot of the prediction intervals for each different conformal
    method.
    """
    # axs.yaxis.set_major_formatter(FormatStrFormatter('%.0f' + "k"))
    # axs.xaxis.set_major_formatter(FormatStrFormatter('%.0f' + "k"))

    lower_bound_ = np.take(lower_bound, num_plots_idx)
    y_pred_sorted_ = np.take(y_pred_sorted, num_plots_idx)
    y_test_sorted_ = np.take(y_test_sorted, num_plots_idx)

    error = y_pred_sorted_-lower_bound_

    warning1 = y_test_sorted_ > y_pred_sorted_+error
    warning2 = y_test_sorted_ < y_pred_sorted_-error
    warnings = warning1 + warning2
    axs.errorbar(
        y_test_sorted_[~warnings],
        y_pred_sorted_[~warnings],
        yerr=np.abs(error[~warnings]),
        capsize=5, marker="o", elinewidth=2, linewidth=0,
        label="Inside prediction interval"
        )
    axs.errorbar(
        y_test_sorted_[warnings],
        y_pred_sorted_[warnings],
        yerr=np.abs(error[warnings]),
        capsize=5, marker="x", elinewidth=2, linewidth=0, color="red",
        label="Outside prediction interval"
        )
    axs.scatter(
        y_test_sorted_[warnings],
        y_test_sorted_[warnings],
        marker="*", color="green",
        label="True value"
    )
    axs.set_xlabel("True Percent Change of LEDD")
    axs.set_ylabel("Prediction of Percent Change of LEDD")
    lims = [
    np.min([axs.get_xlim(), axs.get_ylim()]),  # min of both axes
    np.max([axs.get_xlim(), axs.get_ylim()]),  # max of both axes
]
    # lims = [-1,1]
    ab = AnnotationBbox(
    TextArea(
        f"Coverage: {np.round(coverage, round_to)}\n"
        + f"Interval width: {np.round(width, round_to)}"
    ),
    xy=(lims[0] * 0.9, lims[1] * 0.9),  # Position annotation more relative to axis limits
    xycoords='data',
    boxcoords="offset points",
    frameon=True
    )
    axs.grid(True)
    axs.set_xlim(lims)
    axs.set_ylim(lims)
    axs.plot(lims, lims, '--', alpha=0.75, color="black", label="x=y")
    axs.add_artist(ab)
    axs.set_title(title, fontweight='bold')

In [ ]:
def conformal_prediction_zero_inflated_mapie(X, y_binary, y_continuous, alpha_tilda=0.9):
    """
    Two-stage conformal prediction with multiple MAPIE strategies
    """
    # Split data into five parts
    (X_train, X_calib1, X_calib2, X_calib3, X_test,
     y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
     y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test) = split_into_five_equal_parts(
        X, y_binary, y_continuous
    )
    
    # Define MAPIE strategies like in your original code
    STRATEGIES = {
        "naive": {"method": "naive"},
        "cv_plus": {"method": "plus", "cv": 10},
        "jackknife_plus_ab": {"method": "plus", "cv": Subsample(n_resamplings=50)},
    }
    
    # Initialize dictionaries to store results
    y_pred, y_pis = {}, {}
    y_test_sorted, y_pred_sorted, lower_bound, upper_bound = {}, {}, {}, {}
    coverage, width = {}, {}
    
    # First stage: Classification model (common for all strategies)
    classifier = xgb.XGBClassifier(**binary_params)
    classifier.fit(X_train, y_binary_train)
    
    # Get probabilities for calibration set 1
    probs_calib1 = classifier.predict_proba(X_calib1)[:, 1]
    
    # For each MAPIE strategy
    for strategy, params in STRATEGIES.items():
        # Initialize regressor
        regressor = xgb.XGBRegressor(**regression_params_short)
        
        # Initialize MAPIE with current strategy
        mapie = MapieRegressor(
            estimator=regressor,
            random_state=42,
            **params
        )
        
        # Try different r values
        r_values = np.arange(0.1, 0.95, 0.05)
        all_interval_lengths = []
        
        for r in r_values:
            # First calibration: threshold for zero/non-zero
            alpha_r = np.quantile(probs_calib1, r)
            
            # Second calibration: accuracy of zero predictions
            probs_calib2 = classifier.predict_proba(X_calib2)[:, 1]
            pred_zeros = probs_calib2 <= alpha_r
            beta_hat = np.mean(y_binary_calib2[pred_zeros] == 0)
            
            # Calculate final quantile and ensure it's in [0,1]
            final_quantile = (alpha_tilda - beta_hat * r) / (1 - r)
            final_quantile = np.clip(final_quantile, 0.01, 0.99)
            
            # Third calibration: Use MAPIE for interval width
            probs_calib3 = classifier.predict_proba(X_calib3)[:, 1]
            nonzero_mask = probs_calib3 > alpha_r
            
            if not any(nonzero_mask):
                continue
            
            try:
                # Fit MAPIE on non-zero data
                X_calib3_nonzero = X_calib3[nonzero_mask]
                y_calib3_nonzero = y_cont_calib3[nonzero_mask]
                
                mapie.fit(X_calib3_nonzero, y_calib3_nonzero)
                _, y_pis_calib = mapie.predict(X_calib3_nonzero, alpha=final_quantile)
                
                # Calculate interval width
                interval_width = np.mean(y_pis_calib[:, 1, 0] - y_pis_calib[:, 0, 0])
                all_interval_lengths.append((r, interval_width, alpha_r, mapie))
                
            except Exception as e:
                print(f"Skipping {strategy} with r={r} due to error: {e}")
                continue
        
        if not all_interval_lengths:
            print(f"No valid intervals found for strategy {strategy}")
            continue
        
        # Choose best r value and corresponding MAPIE model
        best_r, best_width, best_alpha_r, best_mapie = min(all_interval_lengths, key=lambda x: x[1])
        
        # Final predictions on test set
        test_probs = classifier.predict_proba(X_test)[:, 1]
        nonzero_mask_test = test_probs > best_alpha_r
        
        # Initialize bounds
        lower_bound[strategy] = np.zeros(len(X_test))
        upper_bound[strategy] = np.zeros(len(X_test))
        
        if any(nonzero_mask_test):
            _, y_pis_test = best_mapie.predict(X_test[nonzero_mask_test], alpha=alpha_tilda)
            lower_bound[strategy][nonzero_mask_test] = y_pis_test[:, 0, 0]
            upper_bound[strategy][nonzero_mask_test] = y_pis_test[:, 1, 0]
        
        # Calculate coverage and width using MAPIE metrics
        coverage[strategy] = regression_coverage_score(
            y_cont_test,
            lower_bound[strategy],
            upper_bound[strategy]
        )
        width[strategy] = regression_mean_width_score(
            lower_bound[strategy],
            upper_bound[strategy]
        )
    
    return lower_bound, upper_bound, y_cont_test, coverage, width

# Run the analysis
results = conformal_prediction_zero_inflated_mapie(
    X=X,
    y_binary=y_pred_all,
    y_continuous=normalized_pctg_change,
    alpha_tilda=0.9
)

lower_bound, upper_bound, y_test, coverage, width = results

# Print results for each strategy
for strategy in coverage.keys():
    print(f"\nStrategy: {strategy}")
    print(f"Coverage: {coverage[strategy]:.3f}")
    print(f"Average interval width: {width[strategy]:.3f}")

# Visualization similar to your original code
rng = np.random.default_rng(21)
perc_obs_plot = 0.90
num_plots = rng.choice(len(y_test), int(perc_obs_plot*len(y_test)), replace=False)

fig, axs = plt.subplots(1, 3, figsize=(18, 6))
coords = axs.ravel()

for strategy, coord in zip(lower_bound.keys(), coords):
    plot_prediction_intervals(
        strategy, coord, y_test, (lower_bound[strategy] + upper_bound[strategy])/2,
        lower_bound[strategy], upper_bound[strategy],
        coverage[strategy], width[strategy],
        num_plots
    )
    coord.grid(True, linestyle='--', alpha=0.4, linewidth=0.8, color='black')

plt.tight_layout()
plt.show()

/opt/conda/lib/python3.10/site-packages/mapie/utils.py:598: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/mapie/utils.py:598: UserWarning: WARNING: The predictions are ill-sorted.
  warnings.warn(
